In [1]:
import os 
%pwd

'c:\\Users\\akg96\\Desktop\\Github\\Text-summarization\\research'

In [2]:
%pwd

'c:\\Users\\akg96\\Desktop\\Github\\Text-summarization\\research'

In [3]:
!dir

 Volume in drive C is Windows
 Volume Serial Number is BCAC-3A53

 Directory of c:\Users\akg96\Desktop\Github\Text-summarization\research

09/18/2023  07:11 AM    <DIR>          .
09/18/2023  07:36 AM    <DIR>          ..
09/18/2023  07:41 AM             6,242 data_ingestion.ipynb
09/18/2023  07:14 AM             1,812 g_down.ipynb
09/16/2023  10:34 AM    <DIR>          logs
05/09/2023  02:13 AM           530,165 samsum-test.csv
05/09/2023  02:13 AM         9,401,418 samsum-train.csv
05/09/2023  02:13 AM           512,131 samsum-validation.csv
09/18/2023  01:39 AM    <DIR>          samsum_dataset
09/18/2023  07:11 AM         7,903,594 summarizer-data.zip
09/13/2023  03:44 AM           203,832 Text_Summarizer_Pegasus.ipynb
09/11/2023  09:54 AM             1,967 trials.ipynb
               8 File(s)     18,561,161 bytes
               4 Dir(s)  125,223,944,192 bytes free


In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\akg96\\Desktop\\Github\\Text-summarization'

In [6]:
# Entity 
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file: Path
    unzip_dir : Path


In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir = config.unzip_dir 
        )

        return data_ingestion_config


In [9]:
#!pip install gdown#

In [10]:
import os 
import urllib.request as request
import gdown
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):

        if not os.path.exists(self.config.local_data_file):

            filename, headers = request.urlretrieve(
            url = self.config.source_URL,
            filename= self.config.local_data_file
            )

            logger.info(f"{filename} download! with following info: \n {headers}")

        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")


    def google_drive_down(self):
        
        if not os.path.exists(self.config.local_data_file):

            prefix = 'https://drive.google.com/uc?/export=download&id='

            url = self.config.source_URL
            output = self.config.local_data_file
            
            file_id = url.split('/')[-2]

            gdown.download(prefix+file_id, output)

            logger.info(f"{output} download! with following info: \n")

        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

        


    def extract_zip_file(self):

        """
        zip_file_path:str 
        Extracts the zip file into the data directory
        Function returns None 
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [12]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.google_drive_down()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

<_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'>
{'artifacts_root': 'artifacts', 'data ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'url = "https://drive.google.com/file/d/15SQdQy-TkE4A68e-hLX_4WOtzsOU7dMw/view?usp=sharing"', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2023-09-18 07:41:26,159: INFO: common: yaml file: config\config.yaml loaded successfully]
<_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'>
{'key1': 'value1', 'key2': 'value2'}
[2023-09-18 07:41:26,167: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-18 07:41:26,167: INFO: common: created directory at: artifacts]
[2023-09-18 07:41:26,179: INFO: common: created directory at: artifacts/data_ingestion]


Downloading...
From: https://drive.google.com/uc?/export=download&id=15SQdQy-TkE4A68e-hLX_4WOtzsOU7dMw
To: c:\Users\akg96\Desktop\Github\Text-summarization\artifacts\data_ingestion\data.zip
100%|██████████| 7.90M/7.90M [00:01<00:00, 3.97MB/s]

[2023-09-18 07:41:42,851: INFO: 2834993816: artifacts/data_ingestion/data.zip download! with following info: 
]
